In [5]:
import numpy as np
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [29]:
bond_ratings_df=pd.read_csv('bond_ratings.csv').sort_index(axis=1)
fund_allocations_df=pd.read_csv('fund_allocations.csv').sort_index(axis=1)
fund_config_df=pd.read_csv('fund_config.csv').sort_index(axis=1)
fund_ratios_df=pd.read_csv('fund_ratios.csv').sort_index(axis=1)
fund_specs_df=pd.read_csv('fund_specs.csv').sort_index(axis=1)
other_specs_df=pd.read_csv('other_specs.csv').sort_index(axis=1)
return_10year_df=pd.read_csv('return_10year.csv').sort_index(axis=1)
return_3year_df=pd.read_csv('return_3year.csv').sort_index(axis=1)
return_5year_df=pd.read_csv('return_5year.csv').sort_index(axis=1)

In [30]:
tagdfs=[bond_ratings_df,other_specs_df,return_3year_df,return_5year_df,fund_specs_df] #since these dataframes contain 'tag' as id.
tag_df = reduce(lambda left,right: pd.merge(left,right,on='tag'), tagdfs)
tag_df=tag_df.drop(columns=['greatstone_rating_y'])
tag_df.rename(columns={'greatstone_rating_x':'greatstone_rating'}, inplace=True)

In [31]:
id_df=pd.merge(fund_config_df,return_10year_df,on="fund_id") #since these dfs contain 'fund_id' as id
final_df=pd.merge(id_df,fund_ratios_df,on="fund_id")

In [32]:
data_df=pd.merge(final_df,tag_df,on="tag")
fund_allocations_df.rename(columns={'id':'tag'},inplace=True) #since column name is 'id' which is nothing but the 'tag'
data_df=pd.merge(data_df,fund_allocations_df,on="tag")

In [33]:
id_tag_dict = dict(zip(data_df.tag, data_df.fund_id))
tag_id_dict=dict(zip(data_df.fund_id,data_df.tag))

In [34]:
data_df.columns[data_df.isna().sum()>3000]

Index(['10_years_alpha_fund', '10_years_beta_fund',
       '10_years_return_mean_annual_fund', '10years_fund_r_squared',
       '10years_fund_std', '10yrs_sharpe_ratio_fund',
       '10yrs_treynor_ratio_fund', 'duration_bond', 'maturity_bond',
       '2010_return_category', '2010_return_fund', '2011_return_category',
       '2011_return_fund', '2012_fund_return', '2012_return_category',
       '2013_category_return', '2013_return_fund', '2014_category_return',
       '2014_return_fund', '2015_return_fund', '2016_return_category',
       '2016_return_fund', 'category_return_2015', 'greatstone_rating',
       '5_years_alpha_fund', '5_years_beta_fund',
       '5_years_return_mean_annual_fund', '5years_fund_r_squared',
       '5years_fund_std', '5yrs_sharpe_ratio_fund', '5yrs_treynor_ratio_fund'],
      dtype='object')

In [35]:
data_df.drop(columns=['10_years_alpha_fund', '10_years_beta_fund',
       '10_years_return_mean_annual_fund', '10years_fund_r_squared',
       '10years_fund_std', '10yrs_sharpe_ratio_fund',
       '10yrs_treynor_ratio_fund', 'duration_bond', 'maturity_bond',
       '2010_return_category', '2010_return_fund', '2011_return_category',
       '2011_return_fund', '2012_fund_return', '2012_return_category',
       '2013_category_return', '2013_return_fund', '2014_category_return',
       '2014_return_fund', '2015_return_fund', '2016_return_category',
       '2016_return_fund', 'category_return_2015',
       '5_years_alpha_fund', '5_years_beta_fund',
       '5_years_return_mean_annual_fund', '5years_fund_r_squared',
       '5years_fund_std', '5yrs_sharpe_ratio_fund', '5yrs_treynor_ratio_fund'],inplace=True)

In [36]:
data_df.drop(columns=['fund_name','parent_company','inception_date','fund_id'], inplace=True)

In [37]:
data_df.columns[data_df.dtypes=='O']

Index(['category', 'mmc_x', 'pc_ratio_x', 'pe_ratio_x', 'ps_ratio_x', 'mmc_y',
       'pc_ratio_y', 'pe_ratio_y', 'ps_ratio_y', '3yrs_treynor_ratio_fund',
       'currency', 'fund_size', 'investment_class'],
      dtype='object')

In [38]:
data_df[['mmc_x', 
'pc_ratio_x', 
'pe_ratio_x', 
'ps_ratio_x', 
'mmc_y',
'pc_ratio_y', 
'pe_ratio_y', 
'ps_ratio_y', 
'3yrs_treynor_ratio_fund']]=data_df[['mmc_x', 
'pc_ratio_x', 
'pe_ratio_x', 
'ps_ratio_x', 
'mmc_y',
'pc_ratio_y', 
'pe_ratio_y', 
'ps_ratio_y', 
'3yrs_treynor_ratio_fund']].apply(lambda x: x.str.replace(',', '').astype(float), axis=1)

In [39]:
#Label Encoder -
le = preprocessing.LabelEncoder()
data_df['currency']=le.fit_transform(data_df['currency'].fillna(method='ffill'))
data_df['fund_size']=le.fit_transform(data_df['fund_size'].fillna(method='ffill'))
data_df['investment_class']=le.fit_transform(data_df['investment_class'].fillna(method='ffill'))
data_df['category']=le.fit_transform(data_df['category'].fillna(method='ffill'))

In [40]:
pred_df=data_df[data_df['greatstone_rating'].isna()]
pred_df.drop(columns=['greatstone_rating'],inplace=True)

data_df=data_df[~data_df['greatstone_rating'].isna()]
data_df=data_df.fillna(value=data_df.mean())
pred_df=pred_df.fillna(value=pred_df.mean())

X=data_df.drop(columns=['greatstone_rating'])
y=data_df['greatstone_rating']


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [46]:
X.shape

(20000, 95)

In [47]:
y.shape

(20000,)

In [41]:
from sklearn.model_selection import train_test_split
# test_size: what proportion of original data is used for test set
train_img, test_img, train_lbl, test_lbl = train_test_split( X, y, test_size=0.2,random_state=82)

In [42]:
from sklearn.ensemble import RandomForestClassifier
rfcl = RandomForestClassifier(n_estimators=300) ### -- no model passed
rfcl = rfcl.fit(train_img,train_lbl)
print("Random Forest model accuracy for training set is: ",rfcl.score(train_img , train_lbl))
print("Random Forest model accuracy for test set is: ",rfcl.score(test_img , test_lbl))

Random Forest model accuracy for training set is:  1.0
Random Forest model accuracy for test set is:  0.776


In [43]:
submission_df=pd.read_csv('sample_submission.csv')
submission_df=submission_df.replace({"fund_id": tag_id_dict})
submission_df=submission_df.rename(columns={"fund_id":"tag"}).drop(columns=['greatstone_rating'])
#Predictions
pred_df['greatstone_rating']=rfcl.predict(pred_df)
df=pred_df[['tag','greatstone_rating']]
submission_df=pd.merge(submission_df,df,on="tag")
submission_df=submission_df.replace({"tag": id_tag_dict}).rename(columns={"tag":"fund_id"}).set_index('fund_id')
submission_df.to_csv('submit.csv')

In [44]:
submission_df.head()

,greatstone_rating
fund_id,
85a6edf9-db47-490c-981a-168ce90434bb,4.0
654dac08-5e5d-4cf0-870e-3167087de7d2,2.0
75214019-f876-42a9-a316-328b759ac4ba,4.0
1fe8de6d-cd33-489e-a829-211f1f622eed,0.0
bd87a7a0-6ca5-4607-a0cf-d396ecd6037a,3.0
